In [ ]:
import plotly.express as px
from appgeopy import *
from my_packages import *

In [ ]:
gwl_h5 = "20240828_GWL_CRFP_model.h5"
gwl_data, gwl_metadata = h5pytools.open_HDF5(gwl_h5)
gwl_stations = [x for x in gwl_data.keys() if x != "date"]
date_arr = pd.to_datetime([x.decode("utf-8") for x in gwl_data["date"]])
len(gwl_stations), gwl_stations

In [ ]:
select_station = "HUATAN"
gwldata_byStation = gwl_data[select_station]
available_wells = sorted(gwldata_byStation.keys())
select_well = available_wells[0] #"07080111"
gwldata_byWell = gwldata_byStation[select_well]
gwl_raw_arr = np.array(gwldata_byWell["measure"]["daily_value"])
gwl_model_arr = np.array(gwldata_byWell["measure"]["model"])

gwl_ts = pd.Series(data=gwl_model_arr, index=date_arr, name="model")
gwl_ts = gwl_ts.loc[gwl_ts.first_valid_index() : gwl_ts.last_valid_index()]

gwl_trend, coeffs = get_polynomial_trend(gwl_ts, order=2)
gwl_detrend = gwl_ts - gwl_trend

px.line(gwl_ts)


In [ ]:
analysis.find_seasonality(gwl_ts)[["Period (days)"]]

In [ ]:
from prophet import Prophet

gwl_ts = pd.Series(data=gwl_model_arr, index=date_arr, name="model")
gwl_ts = gwl_ts.loc[gwl_ts.first_valid_index() : gwl_ts.last_valid_index()]

df = pd.DataFrame(gwl_ts).rename_axis("ds").rename({"model":"y"}, axis=1)
df = df.reset_index()

# 2) Initialize the model
model = Prophet(
    yearly_seasonality=True,   # Enable built-in yearly seasonality
    weekly_seasonality=False,
    daily_seasonality=False,
    seasonality_mode="additive",
)

model.add_seasonality(name='6_years', period=365*6, fourier_order=10)
model.add_seasonality(name='2_years', period=365*2, fourier_order=10)
model.add_seasonality(name='onehalf', period=365*1.5, fourier_order=10)
model.add_seasonality(name='half', period=180, fourier_order=10)
model.add_seasonality(name='quarter', period=90, fourier_order=10)

model.fit(df)

In [ ]:
# 4) Create a future dataframe to forecast (if you want forecasting)
#    or just use the existing data to see the fitted components
future = model.make_future_dataframe(periods=30, freq='D')  # 1-year forecast
forecast = model.predict(future)
fit_df = forecast.set_index("ds").loc[gwl_ts.index]

rmse = root_mean_squared_error(y_true=gwl_ts.values, y_pred=fit_df['yhat'].values)
mae = mean_absolute_error(y_true=gwl_ts.values, y_pred=fit_df['yhat'].values)
r2 = r2_score(y_true=gwl_ts.values, y_pred=fit_df['yhat'].values)
print(f"RMSE: {rmse:.3f}\tMAE={mae:.3f}\tR2={r2:.3f}")

In [ ]:
from prophet.plot import plot_plotly
plot_plotly(model, forecast, figsize=(1500, 800), trend=True, uncertainty=True)

model.add_seasonality(name='half', period=180, fourier_order=15)
model.add_seasonality(name='quarter', period=90, fourier_order=15)
RMSE: 0.16	MAE=0.12	R2=0.783

model.add_seasonality(name='half', period=180, fourier_order=15)
model.add_seasonality(name='quarter', period=90, fourier_order=15)
model.add_seasonality(name='45_day', period=45, fourier_order=15
RMSE: 0.160	MAE=0.123	R2=0.782


